In [ ]:
!pip install -q torch==2.2.1 torchvision torchaudio -f https://download.pytorch.org/whl/torch_stable.html
!pip uninstall transformers -y  # Remove the existing installation
!pip install git+https://github.com/huggingface/transformers.git@main
!pip install -q ipywidgets
!pip install -q accelerate
!pip install -q jupyter
!pip install -q 'bitsandbytes-cuda113'  # Crrect bitsandbytes version for CUDA 11.3
!pip install Bio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 GB 848.1 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 MB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.7/65.7 MB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 MB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 95.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 62.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 50.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 21.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.7/65.7 MB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 MB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 61.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 53.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
!nvcc --version
!pip install -q bitsandbytes

import json, urllib
!gdown 1JowBt1I-ljvkWdaBh0QE7oNMGbfkkAC5

!gdown 19ekDpipQhBERlMLKVQaS6G6LGHp4WAlw

import bitsandbytes as bnb
bnb.__version__  #Ensures CUDA is imported

# Check if CUDA is available in PyTorch, so that bitsandbytes is set up correctly.
import torch
print("CUDA available:", torch.cuda.is_available())
import transformers
help(transformers)

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 7.0 MB/s eta 0:00:00
Downloading...
From: https://drive.google.com/uc?id=1JowBt1I-ljvkWdaBh0QE7oNMGbfkkAC5
To: /content/training12b_new.json
100% 39.9M/39.9M [00:00<00:00, 44.6MB/s]
Downloading...
From: https://drive.google.com/uc?id=19ekDpipQhBERlMLKVQaS6G6LGHp4WAlw
To: /content/training12b_yesno.json
100% 13.6M/13.6M [00:00<00:00, 26.1MB/s]


CUDA available: False


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
No CUDA runtime is found, using CUDA_HOME='/usr/local/cuda'
/usr/local/lib/python3.10/dist-packages/transformers/deepspeed.py:24: FutureWarning: transformers.deepspeed module is deprecated and will be removed in a future version. Please import deepspeed modules directly from transformers.integrations
  warnings.warn(


In [ ]:
import json
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from torch.utils.data import DataLoader, TensorDataset
from Bio import Entrez

def load_data(filepath):
    with open(filepath, 'r') as file:
        data = json.load(file)
    return data['questions']

def prepare_data(tokenizer, questions, batch_size=16):
    bodies = [q['body'] for q in questions]
    input_encodings = tokenizer(bodies, padding=True, truncation=True, max_length=128, return_tensors="pt")
    inputs, masks = input_encodings['input_ids'], input_encodings['attention_mask']
    dataset = TensorDataset(inputs, masks)
    loader = DataLoader(dataset, batch_size=batch_size)
    return loader

def setup_model(model_id):
    device = "cuda" if torch.cuda.is_available() else "cpu"
    tokenizer = AutoTokenizer.from_pretrained(model_id)
    model = AutoModelForSequenceClassification.from_pretrained(model_id)
    model.to(device)
    return model, tokenizer, device

def search_pubmed(query, retmax=9):
    Entrez.email = "jonascalas17@gmail.com"
    handle = Entrez.esearch(db="pubmed", term=query, retmax=retmax)
    record = Entrez.read(handle)
    handle.close()
    return record["IdList"]

def fetch_details(pubmed_ids):
    ids = ",".join(pubmed_ids)
    handle = Entrez.efetch(db="pubmed", id=ids, retmode="xml")
    records = Entrez.read(handle)
    handle.close()
    return records

def extract_snippets(records):
    snippets = []
    for article in records['PubmedArticle']:
        try:
            pmid = article['MedlineCitation']['PMID']
            article_title = article['MedlineCitation']['Article']['ArticleTitle']
            abstract = article['MedlineCitation']['Article']['Abstract']['AbstractText'][0]
            snippets.append({'document': f"http://www.ncbi.nlm.nih.gov/pubmed/{pmid}",
                             'text': abstract})
        except KeyError:
            continue
    return snippets

def generate_output(questions, model, device, loader):
    results = []
    model.eval()
    with torch.no_grad():
        for batch, question in zip(loader, questions):
            inputs, masks = batch
            inputs, masks = inputs.to(device), masks.to(device)
            outputs = model(inputs, attention_mask=masks)
            predictions = outputs.logits.argmax(dim=1).tolist()

            for prediction, q in zip(predictions, questions):
                pubmed_ids = search_pubmed(q['body']) if q['type'] in ['factoid', 'list', 'yesno'] else []
                records = fetch_details(pubmed_ids) if pubmed_ids else None
                snippets = extract_snippets(records) if records else []

                result = {
                    "type": q['type'],
                    "body": q['body'],
                    "id": q['id'],
                    "ideal_answer": "", # this field will be filled when combined with an QA model
                    "exact_answer": [], # this field will be filled when combined with an QA model
                    "documents": [f"http://www.ncbi.nlm.nih.gov/pubmed/{id}" for id in pubmed_ids] if pubmed_ids else [],
                    "snippets": snippets if q['type'] in ['factoid', 'list', 'yesno'] else []
                }
                results.append(result)
    return results

def main():
    model_id = "bionlp/bluebert_pubmed_mimic_uncased_L-12_H-768_A-12"
    model, tokenizer, device = setup_model(model_id)
    questions = load_data('test4.json')  # modify file for other test case
    loader = prepare_data(tokenizer, questions)
    results = generate_output(questions, model, device, loader)

    with open('results.json', 'w') as f:
        json.dump({"questions": results}, f, indent=4)

if __name__ == "__main__":
    main()
